In [ ]:
from agents import Agent, ModelSettings, TResponseInputItem, Runner, RunConfig, trace
from openai.types.shared.reasoning import Reasoning
from pydantic import BaseModel
import asyncio

my_agent_italian = Agent(
  name="My agent",
  instructions="translate the text from english to italian",
  model="gpt-5-nano",
  model_settings=ModelSettings(
    store=True,
    reasoning=Reasoning(
      effort="medium"
    )
  )
)


my_agent_norwegian = Agent(
  name="My agent",
  instructions="translate the text from english to norwegian",
  model="gpt-5-nano",
  model_settings=ModelSettings(
    store=True,
    reasoning=Reasoning(
      effort="medium"
    )
  )
)

class WorkflowInput(BaseModel):
  input_as_text: str


# Main code entrypoint
async def run_workflow(workflow_input: WorkflowInput):
  with trace("New agent"):
    state = {

    }
    workflow = workflow_input.model_dump()
    conversation_history: list[TResponseInputItem] = [
      {
        "role": "user",
        "content": [
          {
            "type": "input_text",
            "text": workflow["input_as_text"]
          }
        ]
      }
    ]
    my_agent_result_temp = await Runner.run(
      my_agent_italian,
      input=[
        *conversation_history
      ],
      run_config=RunConfig(trace_metadata={
        "__trace_source__": "agent-builder",
        "workflow_id": "wf_6999f5a59ce081908c90a7be4ec498ee0610a5b2e27c4494"
      })
    )

    conversation_history.extend([item.to_input_item() for item in my_agent_result_temp.new_items])

    my_agent_result = {
      "output_text": my_agent_result_temp.final_output_as(str)
    }

    return my_agent_result


In [ ]:
input = WorkflowInput(input_as_text="Hello, my name is Gian Marco and I am a developer.")

result = await run_workflow(input)

print(result["output_text"])

AttributeError: 'NoneType' object has no attribute 'final_output'

In [13]:
from agents import Agent, ModelSettings, TResponseInputItem, Runner, RunConfig, trace
from openai.types.shared.reasoning import Reasoning
from pydantic import BaseModel
import asyncio

# 1. Define the Orchestrator Agent
orchestrator_agent = Agent(
  name="Orchestrator",
  instructions="""Determine which language the user wants to translate their text into. 
  Reply strictly with exactly one word: 'italian' or 'norwegian'. 
  If it's unclear, default to 'italian'.""",
  model="gpt-5-nano",
  model_settings=ModelSettings(store=False) 
)

my_agent_italian = Agent(
  name="Italian Translator",
  instructions="translate the text from english to italian",
  model="gpt-5-nano",
  model_settings=ModelSettings(store=True, reasoning=Reasoning(effort="medium"))
)

my_agent_norwegian = Agent(
  name="Norwegian Translator",
  instructions="translate the text from english to norwegian",
  model="gpt-5-nano",
  model_settings=ModelSettings(store=True, reasoning=Reasoning(effort="medium"))
)

class WorkflowInput(BaseModel):
  input_as_text: str

# Main code entrypoint
async def run_workflow(workflow_input: WorkflowInput):
  with trace("New agent"):
    workflow = workflow_input.model_dump()
    conversation_history: list[TResponseInputItem] = [
      {
        "role": "user",
        "content": [
          {
            "type": "input_text",
            "text": workflow["input_as_text"]
          }
        ]
      }
    ]

    # --- STEP 1: Run the Orchestrator ---
    orchestrator_result = await Runner.run(
      orchestrator_agent,
      input=[*conversation_history]
    )
    
    # Extract the routing decision and clean up the text
    decision = orchestrator_result.final_output_as(str).strip().lower()

    # --- STEP 2: Route to the correct agent ---
    if "norwegian" in decision:
        selected_agent = my_agent_norwegian
    else:
        selected_agent = my_agent_italian

    # --- STEP 3: Run the selected agent ---
    my_agent_result_temp = await Runner.run(
      selected_agent,
      input=[*conversation_history],
      run_config=RunConfig(trace_metadata={
        "__trace_source__": "agent-builder",
        "workflow_id": "wf_6999f5a59ce081908c90a7be4ec498ee0610a5b2e27c4494"
      })
    )

    conversation_history.extend([item.to_input_item() for item in my_agent_result_temp.new_items])

    return {
      "routed_to": selected_agent.name, 
      "output_text": my_agent_result_temp.final_output_as(str)
    }



In [14]:
input = WorkflowInput(input_as_text="Please translate this sentence in norwegian: Hello, my name is Gian Marco and I am a developer.")

result = await run_workflow(input)

print(result["output_text"])

Hei, jeg heter Gian Marco og jeg er en utvikler.


In [22]:
from agents import Agent, ModelSettings, TResponseInputItem, Runner, RunConfig, trace, function_tool

class WorkflowInput(BaseModel):
    file_path: str
    instruction: str

import pandas as pd

@function_tool
def calculate_excel_mean(file_path: str) -> str:
    """
    Reads a single-column Excel file and calculates the mathematical mean of its values.
    
    Args:
        file_path (str): The absolute or relative path to the .xlsx file.
        
    Returns:
        str: A sentence containing the calculated mean, or an error message if it fails.
    """
    try:
        
        df = pd.read_excel(file_path)
        first_column = df.columns[0]
        mean_value = df[first_column].mean()
        
        return f"The mean of the column '{first_column}' is {mean_value}."
    
    except FileNotFoundError:
        return f"Error: Could not find an Excel file at the path '{file_path}'."
    except Exception as e:
        return f"Error processing the Excel file: {str(e)}"

excel_agent = Agent(
    name="Excel Handler",
    instructions="You handle Excel files. Use your tool to read the file and execute the user's mathematical instruction. Don't suggest more things to do, strictly reply with what the user asks for.",
    model="gpt-5-nano",
    model_settings=ModelSettings(store=True, reasoning=Reasoning(effort="high")),
    tools=[calculate_excel_mean] # Give the agent the ability to read the file
)

text_agent = Agent(
    name="Text Handler",
    instructions="You handle standard text requests.",
    model="gpt-5-nano"
)

orchestrator_agent = Agent(
    name="Orchestrator",
    instructions="""You are a routing assistant. Look at the user's file and instruction. 
    If they provide an Excel/CSV file AND ask for data analysis or calculations (like finding the mean), reply strictly with the word 'excel'.
    For anything else, reply strictly with the word 'text'.""",
    model="gpt-5-nano",
    model_settings=ModelSettings(store=False) # Keep it fast and cheap
)

async def run_workflow(workflow_input: WorkflowInput):
    with trace("New file workflow"):
        workflow = workflow_input.model_dump()
        
        # 1. Format the memory
        conversation_history: list[TResponseInputItem] = [
            {
                "role": "user",
                "content": [
                    {
                        "type": "input_text",
                        "text": f"File: {workflow['file_path']}\nInstruction: {workflow['instruction']}"
                    }
                ]
            }
        ]

        # --- NEW: The AI Orchestrator makes the decision ---
        orchestrator_result = await Runner.run(
            orchestrator_agent,
            input=[*conversation_history]
        )
        
        # Clean up the output to find the exact word
        decision = orchestrator_result.final_output_as(str).strip().lower()

        # Route based on the AI's understanding of the text
        if "excel" in decision:
            selected_agent = excel_agent
        else:
            selected_agent = text_agent

        # --- Run the selected agent ---
        my_agent_result_temp = await Runner.run(
            selected_agent,
            input=[*conversation_history],
            run_config=RunConfig(trace_metadata={
                "__trace_source__": "agent-builder"
            })
        )

        return {
            "handled_by": selected_agent.name,
            "output": my_agent_result_temp.final_output_as(str)
        }

In [23]:
input = WorkflowInput(instruction="Please take the file and calculate the mean of the first column" , file_path="example.xlsx")

result = await run_workflow(input)

print(result["output"])

2.120082586642795


In [28]:
import pandas as pd
import sys
import io
import traceback
import asyncio
from pydantic import BaseModel
from agents import Agent, ModelSettings, TResponseInputItem, Runner, RunConfig, trace, function_tool
from openai.types.shared.reasoning import Reasoning

# ==========================================
# 0. SETUP: Create a messy test file
# ==========================================
# This creates an Excel file with blanks (visual hierarchy) and a mostly empty column
df_mock = pd.DataFrame({
    "Category": ["Category A", None, None, "Category B", None],
    "Amount": [100, 150, 200, 50, 75],
    "Date Registered": [None, None, "2023-01-01", None, None]
})
df_mock.to_excel("example.xlsx", index=False)


# ==========================================
# 1. BLUEPRINT & STATE
# ==========================================
class WorkflowInput(BaseModel):
    file_path: str
    instruction: str

# Persistent memory for the Python interpreter
_agent_memory_state = {
    "pd": pd  # Pre-load pandas so the agent doesn't have to import it
}


# ==========================================
# 2. THE TOOLS
# ==========================================
@function_tool
def profile_excel_structure(file_path: str) -> str:
    """
    Analyzes an Excel file and returns a summary of its structure, missing values, and a sample of the data.
    Use this FIRST to understand how the user formatted the sheet before trying to clean or calculate anything.
    """
    try:
        df = pd.read_excel(file_path)
        missing_data = df.isnull().sum().to_string()
        sample_data = df.head(15).to_string()
        
        report = (
            f"--- COLUMNS & MISSING VALUES ---\n{missing_data}\n\n"
            f"--- FIRST 15 ROWS (VISUAL LAYOUT) ---\n{sample_data}"
        )
        return report
    except Exception as e:
        return f"Error reading file: {str(e)}"

@function_tool
def execute_python(code_string: str) -> str:
    """
    Executes Python code in a persistent stateful environment. 
    Variables created in one call (like 'df') will be available in the next call.
    ALWAYS use print() to output the final result or preview of the dataframe so you can see it.
    
    Args:
        code_string (str): The Python code to execute.
    """
    # --- ADD THESE THREE LINES ---
    print("\n⚙️ THE AGENT IS RUNNING THIS CODE:")
    print(code_string)
    print("----------------------------------\n")

    old_stdout = sys.stdout
    redirected_output = sys.stdout = io.StringIO()
    
    try:
        exec(code_string, _agent_memory_state)
        output = redirected_output.getvalue().strip()
        
        if not output:
            return "Code executed successfully, but nothing was printed. Did you forget to print() the result?"
        return f"Output:\n{output}"
        
    except Exception:
        error_msg = traceback.format_exc()
        return f"Error executing code:\n{error_msg}"
    finally:
        sys.stdout = old_stdout


# ==========================================
# 3. THE AGENT
# ==========================================
data_analyst_agent = Agent(
    name="Data Analyst",
    instructions="""You are an autonomous Data Analyst. 
    When given a file and an instruction:
    1. ALWAYS run `profile_excel_structure` first to see the data format.
    2. Identify human formatting quirks (like blanks used for visual grouping, or useless mostly-empty columns).
    3. Use `execute_python` to write pandas code that loads the file into a variable named `df`, cleans those quirks, and performs the user's requested instruction.
    4. Provide a final summary of what the file contained, what quirks you fixed, and the final answer to the user's instruction.""",
    model="gpt-5-nano",
    model_settings=ModelSettings(store=True, reasoning=Reasoning(effort="high")),
    tools=[profile_excel_structure, execute_python]
)


# ==========================================
# 4. THE WORKFLOW
# ==========================================
async def run_workflow(workflow_input: WorkflowInput):
    with trace("Data Analysis Workflow"):
        workflow = workflow_input.model_dump()
        
        conversation_history: list[TResponseInputItem] = [
            {
                "role": "user",
                "content": [
                    {
                        "type": "input_text",
                        "text": f"File: {workflow['file_path']}\nInstruction: {workflow['instruction']}"
                    }
                ]
            }
        ]

        my_agent_result_temp = await Runner.run(
            data_analyst_agent,
            input=[*conversation_history],
            run_config=RunConfig(trace_metadata={
                "__trace_source__": "agent-builder"
            })
        )

        return {
            "output": my_agent_result_temp.final_output_as(str)
        }

# ==========================================
# 5. EXECUTION
# ==========================================
strict_instruction = """
You must complete the following steps in this exact order:
1. Load the file and clean up the visual blanks and useless columns.
2. CRITICAL: You must use the execute_python tool to run `df.to_excel('cleaned_example.xlsx', index=False)` to save the cleaned dataframe to the disk.
3. Calculate the total sum of the Amount column for each Category.
4. Confirm in your final response that the new file was successfully saved.
"""

input_data = WorkflowInput(
    file_path="example.xlsx", 
    instruction=strict_instruction
)

result = await run_workflow(input_data)
print(result["output"])

# Await the execution (assumes Jupyter Notebook environment)
result = await run_workflow(input_data)
print(result["output"])


⚙️ THE AGENT IS RUNNING THIS CODE:
import pandas as pd
# Load
df = pd.read_excel('example.xlsx')

# Clean: remove columns that are completely empty
df = df.dropna(axis=1, how='all')

# Fix visual blanks in Category by forward filling
if 'Category' in df.columns:
    df['Category'] = df['Category'].ffill()

# Normalize Amount to numeric
if 'Amount' in df.columns:
    df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce')

# Parse Date Registered to datetime if present
if 'Date Registered' in df.columns:
    df['Date Registered'] = pd.to_datetime(df['Date Registered'], errors='coerce')

# Drop any rows that are completely empty
df = df.dropna(how='all')

# Save cleaned data
df.to_excel('cleaned_example.xlsx', index=False)

# Print a quick preview
print('Cleaned DataFrame preview (first 5 rows):')
print(df.head())

# Confirm file saved
import os
print('File saved:', os.path.exists('cleaned_example.xlsx'))

----------------------------------


⚙️ THE AGENT IS RUNNING THIS CODE:
# Ste